<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# Exploratory Data Analysis

This notebook walks you through the steps to exploratory data analysis.

In [1]:
from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
import pickle

In [2]:
df = pd.DataFrame()

In [3]:
#list_df=['0_to_25','25_to_50','50_to_75','75_to_100']
list_df=['100_to_125','125_to_150','150_to_175','175_to_200']
for value in list_df:
    print('Reading Train df:', value.replace('_',' '))
    #read the pickle file
    picklefile = open('data_train_df_'+value, 'rb')
    #unpickle the dataframe
    df_tmp = pickle.load(picklefile)
    #close file
    picklefile.close()
    df = pd.concat([df,df_tmp])
    print(len(df_tmp))

Reading Train df: 100 to 125
783313
Reading Train df: 125 to 150
560398
Reading Train df: 150 to 175
655101
Reading Train df: 175 to 200
620432


In [4]:
#df

In [5]:
df['label'].unique()

array([4, 2, 1, 3, 7, 6, 8, 5, 9])

In [6]:
#df[df['field_id']==71740]

In [7]:
df=df.reset_index(drop=True)
#df

In [8]:
df['label'].value_counts()

7    914236
2    344026
6    326974
1    317871
4    241486
8    172747
5    124293
3    122761
9     54850
Name: label, dtype: int64

In [9]:
#df[(df['label']>=6.9) & (df['label']<7)]['label']

In [10]:
#df.loc[1517]

In [11]:
#df[df['field_id']==71740]

### Rename columns

In [12]:
datetime=['2017-04-01T00:00:00+0000', '2017-04-11T00:00:00+0000',
       '2017-04-21T00:00:00+0000', '2017-05-01T00:00:00+0000',
       '2017-05-11T00:00:00+0000', '2017-05-21T00:00:00+0000',
       '2017-05-31T00:00:00+0000', '2017-06-10T00:00:00+0000',
       '2017-06-20T00:00:00+0000', '2017-06-30T00:00:00+0000',
       '2017-07-05T00:00:00+0000', '2017-07-10T00:00:00+0000',
       '2017-07-15T00:00:00+0000', '2017-07-20T00:00:00+0000',
       '2017-07-25T00:00:00+0000', '2017-07-30T00:00:00+0000',
       '2017-08-04T00:00:00+0000', '2017-08-09T00:00:00+0000',
       '2017-08-14T00:00:00+0000', '2017-08-19T00:00:00+0000',
       '2017-08-24T00:00:00+0000', '2017-08-29T00:00:00+0000',
       '2017-09-08T00:00:00+0000', '2017-09-18T00:00:00+0000',
       '2017-09-23T00:00:00+0000', '2017-09-28T00:00:00+0000',
       '2017-10-03T00:00:00+0000', '2017-10-08T00:00:00+0000',
       '2017-10-13T00:00:00+0000', '2017-10-18T00:00:00+0000',
       '2017-10-23T00:00:00+0000', '2017-10-28T00:00:00+0000',
       '2017-11-02T00:00:00+0000', '2017-11-07T00:00:00+0000',
       '2017-11-12T00:00:00+0000', '2017-11-17T00:00:00+0000',
       '2017-11-22T00:00:00+0000', '2017-11-27T00:00:00+0000']
bands=['B01', 'B02', 'B03', 'B04', 'B05','B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'CLM']
columns_name=[]
for value in datetime:
    for band in bands:
        columns_name.append(value.replace('T00:00:00+0000','').replace('-','') + "_" + band)

In [13]:
df.columns = columns_name + ['label','field_id']
df

,20170401_B01,20170401_B02,20170401_B03,20170401_B04,20170401_B05,20170401_B06,20170401_B07,20170401_B08,20170401_B09,20170401_B11,...,20171127_B06,20171127_B07,20171127_B08,20171127_B09,20171127_B11,20171127_B12,20171127_B8A,20171127_CLM,label,field_id
0,19.0,21.0,34.0,36.0,49.0,68.0,74.0,82.0,77.0,79.0,...,64.0,70.0,80.0,72.0,77.0,56.0,75.0,0.0,4,7426.0
1,21.0,21.0,34.0,41.0,55.0,73.0,78.0,85.0,88.0,91.0,...,65.0,72.0,76.0,80.0,87.0,63.0,78.0,0.0,4,7426.0
2,21.0,26.0,39.0,47.0,55.0,73.0,78.0,86.0,88.0,91.0,...,65.0,72.0,77.0,80.0,87.0,63.0,78.0,0.0,4,7426.0
3,21.0,28.0,42.0,53.0,61.0,73.0,78.0,86.0,88.0,97.0,...,63.0,72.0,76.0,80.0,95.0,70.0,76.0,0.0,4,7426.0
4,21.0,25.0,34.0,39.0,60.0,77.0,84.0,86.0,88.0,91.0,...,74.0,82.0,85.0,80.0,89.0,64.0,83.0,0.0,4,73914.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619239,56.0,51.0,58.0,71.0,80.0,81.0,83.0,86.0,138.0,80.0,...,69.0,72.0,77.0,78.0,114.0,97.0,76.0,0.0,1,52255.0
2619240,55.0,51.0,58.0,73.0,80.0,82.0,84.0,86.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
2619241,55.0,51.0,59.0,73.0,80.0,82.0,84.0,87.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
2619242,55.0,52.0,60.0,72.0,80.0,82.0,85.0,88.0,134.0,81.0,...,70.0,73.0,78.0,80.0,119.0,104.0,79.0,0.0,1,52255.0


In [14]:
for value in datetime:
    print(value)
    print(df[value.replace('T00:00:00+0000','').replace('-','')+'_'+'CLM'].value_counts())

2017-04-01T00:00:00+0000
255.0    1492370
0.0      1126874
Name: 20170401_CLM, dtype: int64
2017-04-11T00:00:00+0000
0.0      2605293
255.0      13951
Name: 20170411_CLM, dtype: int64
2017-04-21T00:00:00+0000
0.0    2619244
Name: 20170421_CLM, dtype: int64
2017-05-01T00:00:00+0000
255.0    1492424
0.0      1126820
Name: 20170501_CLM, dtype: int64
2017-05-11T00:00:00+0000
255.0    2443988
0.0       175256
Name: 20170511_CLM, dtype: int64
2017-05-21T00:00:00+0000
0.0      2596547
255.0      22697
Name: 20170521_CLM, dtype: int64
2017-05-31T00:00:00+0000
0.0      2344766
255.0     274478
Name: 20170531_CLM, dtype: int64
2017-06-10T00:00:00+0000
255.0    2506934
0.0       112310
Name: 20170610_CLM, dtype: int64
2017-06-20T00:00:00+0000
0.0      2461672
255.0     157572
Name: 20170620_CLM, dtype: int64
2017-06-30T00:00:00+0000
255.0    1696490
0.0       922754
Name: 20170630_CLM, dtype: int64
2017-07-05T00:00:00+0000
0.0      2551535
255.0      67709
Name: 20170705_CLM, dtype: int64
2017-07

In [15]:
bands=['B01', 'B02', 'B03', 'B04', 'B05','B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A']
for value in datetime:
    value = value.replace('T00:00:00+0000','').replace('-','') 
    for band in bands:
        df[value+'_'+band]= np.where(df[value+'_'+'CLM'] == 255, np.nan,df[value+'_'+band])

In [16]:
df.isnull().sum()

20170401_B01    1492370
20170401_B02    1492370
20170401_B03    1492370
20170401_B04    1492370
20170401_B05    1492370
                 ...   
20171127_B12          0
20171127_B8A          0
20171127_CLM          0
label                 0
field_id              0
Length: 496, dtype: int64

In [17]:
list_columns_delete = [value for value in df.columns if value not in ['label']]
df_label = df.groupby('field_id').first().reset_index()
df_label.drop(df_label[list_columns_delete], axis='columns', inplace=True)
df_label

,label
0,9
1,4
2,2
3,4
4,6
...,...
3566,6
3567,7
3568,4
3569,4


In [18]:
df.drop(['label'], axis='columns', inplace=True)
df_median = df.groupby('field_id').median().reset_index()
df_median

,field_id,20170401_B01,20170401_B02,20170401_B03,20170401_B04,20170401_B05,20170401_B06,20170401_B07,20170401_B08,20170401_B09,...,20171127_B04,20171127_B05,20171127_B06,20171127_B07,20171127_B08,20171127_B09,20171127_B11,20171127_B12,20171127_B8A,20171127_CLM
0,57.0,25.0,25.0,37.0,51.0,58.0,64.0,68.0,73.0,68.0,...,62.0,70.0,76.0,81.0,86.0,80.0,115.0,92.0,86.0,0.0
1,99.0,20.0,22.0,32.0,42.0,53.0,63.0,69.0,75.0,81.0,...,38.0,53.0,79.0,89.0,97.0,95.0,103.0,75.0,96.0,0.0
2,106.0,21.0,27.0,39.0,51.0,59.0,65.0,70.0,73.0,77.0,...,36.0,48.0,62.0,68.0,75.0,75.0,89.0,62.0,75.0,0.0
3,121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,36.0,83.0,97.0,103.0,106.0,64.0,37.0,104.0,0.0
4,138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.0,97.0,97.0,101.0,107.0,104.0,135.0,109.0,103.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,122508.0,18.0,25.0,41.0,64.0,77.0,82.0,90.0,98.0,103.0,...,64.0,72.0,75.0,81.0,87.0,89.0,118.0,89.0,88.0,0.0
3567,122532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,66.0,73.0,75.0,78.0,82.0,84.0,118.0,93.0,83.0,0.0
3568,122534.0,10.5,14.0,24.0,34.0,43.0,51.0,56.0,61.0,61.0,...,40.0,53.0,73.0,80.0,88.0,84.0,96.0,65.0,86.0,0.0
3569,122585.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,58.5,80.0,89.0,95.0,92.0,103.5,83.0,94.0,0.0


In [19]:
df_median = pd.concat([df_median,df_label], axis=1)

In [20]:
df_median['label'].unique()

array([9, 4, 2, 6, 3, 1, 7, 8, 5])

In [21]:
df_median.describe()

,field_id,20170401_B01,20170401_B02,20170401_B03,20170401_B04,20170401_B05,20170401_B06,20170401_B07,20170401_B08,20170401_B09,...,20171127_B05,20171127_B06,20171127_B07,20171127_B08,20171127_B09,20171127_B11,20171127_B12,20171127_B8A,20171127_CLM,label
count,3571.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,...,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000,3571.00000,3571.000000,3571.000000,3571.0,3571.000000
mean,60186.215346,17.555996,20.789314,31.712786,43.274148,52.559729,61.747550,67.431638,71.555296,74.698553,...,56.421591,69.747970,76.093671,81.841781,82.611733,102.75413,78.529684,82.370905,0.0,4.273593
std,35007.563919,8.321372,9.698637,12.411902,18.006654,17.956574,16.238864,16.809968,17.630071,18.404151,...,14.658900,12.861454,14.369917,14.980679,13.790123,19.00516,21.033832,14.374542,0.0,2.151262
min,57.000000,0.000000,1.000000,5.000000,5.000000,11.000000,20.000000,22.000000,25.000000,22.000000,...,22.000000,33.000000,37.000000,34.500000,41.000000,50.00000,25.000000,40.000000,0.0,1.000000
25%,29979.500000,12.000000,14.000000,23.000000,29.000000,39.000000,52.000000,56.750000,60.000000,63.000000,...,45.000000,61.000000,66.000000,72.000000,73.000000,89.00000,63.000000,73.000000,0.0,2.000000
50%,60189.000000,17.000000,20.000000,30.000000,41.000000,50.000000,61.000000,68.000000,72.000000,75.000000,...,54.000000,71.000000,76.000000,82.000000,83.000000,104.00000,79.000000,82.000000,0.0,4.000000
75%,89903.000000,22.000000,27.000000,40.000000,56.000000,64.000000,72.000000,77.750000,82.500000,85.000000,...,68.000000,79.000000,86.000000,92.000000,91.000000,117.50000,94.000000,92.000000,0.0,6.000000
max,122649.000000,120.000000,122.500000,123.000000,125.500000,139.000000,146.000000,149.000000,163.500000,227.000000,...,116.000000,124.000000,155.000000,163.000000,138.000000,169.00000,161.000000,155.000000,0.0,9.000000


In [22]:
#create a file
#picklefile = open('df_median_2', 'wb')
#pickle the dataframe
#pickle.dump(df_median, picklefile)
#close file
#picklefile.close()

### Conclusions

- En algunas fechas hay probabilidad de Nubes
- Cuando hay probabilidad de nubes (= 255), se remplaza por valor NaN en las bandas en la fecha correspondiente
- Para calcular los valores NaN utilizamos interpolación de la serie temporal